In [2]:
# Multiclass classification using a neural network

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import seaborn as sns
import pandas as pd


In [3]:
df_training = pd.read_csv('./dataset/datasetC.csv', header=None)
df_testing = pd.read_csv('./dataset/datasetCTest.csv', header=None)

# X is all the data except the last column
X_training = df_training.iloc[:, :-1]

# y is the last column
y_training = df_training.iloc[:, -1]

In [4]:
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [5]:
# Initialization of models
models = {  
    "xgboost": OneVsRestClassifier(XGBClassifier(random_state=42)),
    "SGD" : SGDClassifier(random_state=42),
    "svc" : OneVsRestClassifier(SVC()),
}

from sklearn.model_selection import train_test_split

#y_training = y_training - 1
x_train, x_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2, random_state=42)

# Scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


In [6]:
# saving the scaler for later use
import pickle
pickle.dump(scaler, open("myscaler.pkl", "wb"))

In [7]:
# Train the models
for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(x_train_scaled, y_train)

  #Make predictions
  y_test_pred = model.predict(x_test_scaled)

  # Test set Performance
  accuracy = accuracy_score(y_test, y_test_pred)

  print(list(models.keys())[i])

  print('- Accuracy : {:.4f}'.format(accuracy))

  print('\n')


xgboost
- Accuracy : 0.7970


SGD
- Accuracy : 0.7590


svc
- Accuracy : 0.8330




### Neural Networks

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Convert labels to categorical one-hot encoding
y_train_categorical = to_categorical(y_train)

# Create a Sequential model
model_nn = Sequential()

# Add an input layer and a hidden layer with 20 neurons
model_nn.add(Dense(30, input_dim=x_train_scaled.shape[1], activation='relu'))
model_nn.add(BatchNormalization())

# model_nn.add(Dense(32, input_dim=x_train_scaled.shape[1], activation='relu'))
# model_nn.add(Dense(6, activation='softmax'))  # 6 neurons in the output layer

# Add dropout after the first hidden layer
model_nn.add(Dropout(0.5))

model_nn.add(Dense(20, activation='relu'))
model_nn.add(BatchNormalization())

# Add dropout after the second hidden layer
model_nn.add(Dropout(0.5))

# Add an output layer with a neuron for each class, using softmax activation
model_nn.add(Dense(y_train_categorical.shape[1], activation='softmax'))

# Compile the model
model_nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='loss', patience=10)

# Define learning rate reduction
lr_reduction = ReduceLROnPlateau(monitor='loss', patience=2, verbose=1, factor=0.5, min_lr=0.00001)

# Train the model
model_nn.fit(x_train_scaled, y_train_categorical, epochs=100, batch_size=10, callbacks=[early_stopping, lr_reduction])

# Evaluate the model
loss, accuracy = model_nn.evaluate(x_test_scaled, to_categorical(y_test))

print('Test accuracy:', accuracy)

Epoch 1/100
400/400 [==============================] - 6s 6ms/step - loss: 2.1240 - accuracy: 0.2528 - lr: 0.0010
Epoch 2/100
400/400 [==============================] - 2s 4ms/step - loss: 1.5178 - accuracy: 0.4055 - lr: 0.0010
Epoch 3/100
400/400 [==============================] - 2s 4ms/step - loss: 1.2785 - accuracy: 0.4935 - lr: 0.0010
Epoch 4/100
400/400 [==============================] - 2s 5ms/step - loss: 1.1296 - accuracy: 0.5807 - lr: 0.0010
Epoch 5/100
400/400 [==============================] - 2s 6ms/step - loss: 1.0366 - accuracy: 0.6225 - lr: 0.0010
Epoch 6/100
400/400 [==============================] - 2s 4ms/step - loss: 0.9710 - accuracy: 0.6492 - lr: 0.0010
Epoch 7/100
400/400 [==============================] - 2s 4ms/step - loss: 0.9176 - accuracy: 0.6747 - lr: 0.0010
Epoch 8/100
400/400 [==============================] - 2s 4ms/step - loss: 0.8914 - accuracy: 0.6770 - lr: 0.0010
Epoch 9/100
400/400 [==============================] - 2s 4ms/step - loss: 0.8462 - accu

In [11]:
# save the model
model_nn.save('model_nn.keras')

In [ ]:
from keras.models import load_model

# Load the model
loaded_model = load_model('model_nn.keras')

# New data = df_testing
x_test_new = df_testing

# Scaling the data
scaler = pickle.load(open("myscaler.pkl", "rb"))
x_test_new_scaled = scaler.transform(x_test_new)

# Make predictions on the new data
predictions = loaded_model.predict(x_test_new_scaled)

In [ ]:
# Convert predictions to label indexes using argmax
predicted_labels = np.argmax(predictions, axis=1)

# Add 1 to the predicted labels to get them back in the original format
predicted_labels_original = predicted_labels + 1

# Plot the distribution of predicted classes
plt.hist(predicted_labels_original, bins=np.arange(predictions.shape[1] + 1) - 0.5, edgecolor='black')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.title('Distribution of predicted classes')
plt.show()

In [ ]:
# Save the predicted labels to a numpy file
np.save('labels22.npy', predicted_labels_original)

In [ ]:
# Load the predicted labels
loaded_labels = np.load('labels22.npy')

# Print the first 20 entries of the predicted labels
print(loaded_labels[:400])